***Parse PDF files***

In [30]:
%pip install -r ./requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Unstructured  Partition PDF

In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
loader = UnstructuredPDFLoader("pdf_files/mock_client_slide_deck_lat.pdf")
recomm_deck = loader.load()
print(recomm_deck)

[Document(page_content='Current Themes and Recommendations\n\nPreventative Care\n\nAccess to Care\n\nBehavioral Health\n\nThemes\n\nPreventative\n\nER utilization and urgent care visits are low • Convenience\n\nER utilization and urgent care visits are low • Convenience\n\ncare increased\n\nTelehealth\n\nutilized by 30% of members\n\n\n\n\n\n\n\n30% of members had a behavioral health conditions 35% of behavioral members are dependents 25% of prescriptions dispensed without PCP visit\n\nRecommendations HealthHive\n\nPreventive Care Kit Use HealthHive to schedule and track preventive care appointments.\n\nCareConnect Access Platform\n\nChronic Conditions\n\nObesity and hypertension were top chronic conditions, both are risk factors for Type 2 diabetes ChronicCare Companion Program\n\nObesity and hypertension were top chronic conditions, both are risk factors for Type 2 diabetes ChronicCare Companion Program\n\nPharmacy\n\nSpecialty drugs were a large part of expenses of high cost claims\

Add client_name & year to the Metadata

In [11]:
recomm_deck[0].metadata['client_name'] = 'mock_client'
recomm_deck[0].metadata['year'] = '2024'


In [14]:
print(recomm_deck[0].metadata)

{'source': 'pdf_files/mock_client_slide_deck_lat.pdf', 'client_name': 'mock_client', 'year': '2024'}


Embed & add documents to VectorStore

In [24]:
import chromadb

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()
vector_stor = Chroma.from_documents(recomm_deck, embeddings, client=new_client, collection_name = "openai_collection")


Self Querying Retriever

In [19]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

Create Retriever

In [25]:
metadata_field_info = [
    AttributeInfo(
        name = "source",
        description="Name of the document",
        type="string",
    ),
    AttributeInfo(
        name = "client_name",
        description="Name of the client",
        type="string",
    ),
    AttributeInfo(
        name = "year",
        description="Document created year",
        type="string",
    ),

]
document_content_desc = "mock client recommendations"
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(llm,
                                        vector_stor,
                                        document_content_desc,
                                        metadata_field_info)


In [26]:
print(retriever)

vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x327fa5550> query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\")), lt(\\"length\\", 180), eq

Relevant Documents

In [27]:
get_content = retriever.get_relevant_documents("what are curent themes for client mock_client?")

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


RAG prompt

In [41]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [32]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'],
    template = "You are an assistant for question answering tasks. Use the following pieces of retrieved context to answer the question. if you don't know the answer just say don't know. \nQuestion \nContext: {context} \nAnswer"))])
print(prompt)

input_variables=['context'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question answering tasks. Use the following pieces of retrieved context to answer the question. if you don't know the answer just say don't know. \nQuestion \nContext: {context} \nAnswer"))]


In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
            | rag_prompt
             | llm
             | StrOutputParser()
            )
rag_chain.invoke("What are the Preventive care recommendations suggested for mock_client in 2024?")

OutputParserException: Parsing text
```json
{
    "query": "Preventive care recommendations mock_client",
    "filter": "eq(\"client_name\", \"mock_client\") and eq(\"year\", \"2024\")"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 34.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


[Document(page_content='Current Themes and Recommendations\n\nPreventative Care\n\nAccess to Care\n\nBehavioral Health\n\nThemes\n\nPreventative\n\nER utilization and urgent care visits are low • Convenience\n\nER utilization and urgent care visits are low • Convenience\n\ncare increased\n\nTelehealth\n\nutilized by 30% of members\n\n\n\n\n\n\n\n30% of members had a behavioral health conditions 35% of behavioral members are dependents 25% of prescriptions dispensed without PCP visit\n\nRecommendations HealthHive\n\nPreventive Care Kit Use HealthHive to schedule and track preventive care appointments.\n\nCareConnect Access Platform\n\nChronic Conditions\n\nObesity and hypertension were top chronic conditions, both are risk factors for Type 2 diabetes ChronicCare Companion Program\n\nObesity and hypertension were top chronic conditions, both are risk factors for Type 2 diabetes ChronicCare Companion Program\n\nPharmacy\n\nSpecialty drugs were a large part of expenses of high cost claims\